** not working **  
https://www.kaggle.com/code/rasmus01610/notebook-multi-gpu-training-with-pytorch-lightning/notebook

In [3]:
#!pip install monai
#!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.5 MB/s eta 0:00:00
Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 10.7 MB/s eta 0:00:00
Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)


In [1]:
import os
import PIL
import torch
import numpy as np

from monai.data import DataLoader, ArrayDataset
from torch.optim.lr_scheduler import CosineAnnealingLR
from monai.networks.nets import DenseNet121
from monai.transforms import (
    EnsureChannelFirst,
    AsDiscrete,
    Compose,
    LoadImage,
    ScaleIntensity,
)
import glob
import os

from torch.utils.data import random_split
import pytorch_lightning as pl
from torch import optim
from sklearn.metrics import accuracy_score

In [2]:
#!wget https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/MedNIST.tar.gz
#!tar -zxf MedNIST.tar.gz

In [3]:
path = "./MedNIST"
images = glob.glob(path + '/*/*.jpeg')
labels = [image.split('/')[-2] for image in images]
unique_labels = list(set(labels))
labels = [unique_labels.index(i) for i in labels]

In [4]:
num_classes = len(unique_labels)

train_transforms = Compose(
    [
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        ScaleIntensity(),
    ]
)

y_trans = Compose([AsDiscrete(to_onehot=num_classes)])

In [5]:
ds = ArrayDataset(images, labels=labels, img_transform=train_transforms, label_transform=y_trans)
train_ds, val_ds = random_split(ds, [len(ds) - 12000, 12000])

In [6]:
train_dl = DataLoader(train_ds, shuffle=True, pin_memory=True, num_workers=2, batch_size=256)
val_dl = DataLoader(val_ds, shuffle=False, pin_memory=True, num_workers=2, batch_size=256)

In [7]:
class MedNISTModel(pl.LightningModule):
    def __init__(self, net, lr, loss):
        super().__init__()
        self.net = net
        self.lr = lr
        self.loss = loss 
        
    def forward(self, x):
        return torch.nn.functional.softmax(self.net(x), dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y.float())
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y.float())
        val_acc = (y.argmax(dim=1) == y_hat.argmax(dim=1)).float().sum() / y.shape[0]
        self.log("val_acc", val_acc, prog_bar=True, sync_dist=True)
        self.log("val_loss", loss, prog_bar=True, sync_dist=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(optimizer, self.trainer.max_epochs * 200, 0)
        return [optimizer], [scheduler]

In [8]:
net = DenseNet121(spatial_dims=2, in_channels=1, out_channels=num_classes)
lr = 1e-3
loss = torch.nn.CrossEntropyLoss()
model = MedNISTModel(net, lr, loss)

In [9]:
trainer = pl.Trainer(accelerator="gpu",devices=1,strategy="ddp_notebook", max_epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kotech/venv-lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [10]:
trainer.fit(model, train_dl, val_dl)

Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type             | Params
------------------------------------------
0 | net  | DenseNet121      | 7.0 M 
1 | loss | CrossEntropyLoss | 0     
------------------------------------------
7.0 M     Trainable params
0         Non-tra

Sanity Checking: |                                                                                            …

ProcessExitedException: process 0 terminated with signal SIGSEGV